In [2]:
print('Hola hola!')




Hola hola!


In [5]:
%pip install selenium==4.4.3

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
from selenium import webdriver
driver = webdriver.Chrome('./chromedriver')
#driver = webdriver.Chrome('/Users/IvanJobi/Desktop/chromedriver.exe')
#driver = webdriver.Chrome('/Users/IvanJobi/Desktop/chromedriver')


/var/folders/tl/_gc7cws53slcfj7kmnkdymgr0000gn/T/ipykernel_29603/27914499.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [14]:
driver.get('https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=data%2520science&sc.keyword=data%2520science&locT=&locId=&jobType=')


In [13]:
driver.get('https://facebook.com') # Con este método, llegamos a un URL del internet como en este jemplo: Google

In [15]:
def init_scrapper(driverpath,keyword):
    """"initialize the glassdor scrapper given a job keyword"""
    # Iniciamos el webdriver -> La función principal
    # No olvides que el "driverpath" debe ser la ruta al archivo de chromedriver EN TU COMPU
    driver = webdriver.Chrome(executable_path=driverpath)
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    driver.get(url)

    return driver

In [23]:
path = "./chromedriver"
keyword = "blockchain"
driver = init_scrapper(path,keyword)

/var/folders/tl/_gc7cws53slcfj7kmnkdymgr0000gn/T/ipykernel_29603/2439416938.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driverpath)


In [17]:
from selenium import webdriver
import sys
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

In [19]:
def get_company_overview_data(Company_Overview):
    """"gets the company overview data (dont the name of the func speaks for itself?)"""
    Company_Overview_dict = {"size":"",
                            "founded":"",
                            "type":"",
                            "industry":"",
                            "sector":"",
                            "revenue":"",}

    for i, key in enumerate(Company_Overview_dict):
        try:
            Company_Overview_dict[key] = Company_Overview[i].text
        except:
            # print(f"{key} not found")
            Company_Overview_dict[key] = -1
    return Company_Overview_dict


In [22]:
jobs = driver.find_elements(By.CLASS_NAME,'JobsList_jobListItem__wjTHv') # Súper mucho OJO Aquí 👀
len(jobs)

31

In [41]:
def get_jobs(keyword, num_jobs, verbose, path, slp_time):

    """Gathers jobs as a dataframe, scraped from Glassdoor"""
    driver = init_scrapper(path,keyword)
    jobs = []
    #Let the page load. Change this number based on your internet speed.
    time.sleep(2*slp_time)

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Going through each job in this page
        job_buttons = driver.find_elements(By.CLASS_NAME, "JobsList_jobListItem__wjTHv")  #Acá tenemos que referenciar a los elementos HTML que contienen la info que queremos
        for job_button in job_buttons:

            # print(f"Progress: {len(jobs)}/{num_jobs}")
            if len(jobs) >= num_jobs:
                break
            job_button.click()  #You might
            time.sleep(slp_time)
            # Test for the "Sign Up" prompt and get rid of it.
            # close modal (just appear on the first pass)
            if len(jobs) == 1: # 0?
                try:
                    driver.find_element(By.CLASS_NAME, 'CloseButton').click()
                    print("modal cerrado con exito")
                except NoSuchElementException:
                    print("no hay modal xd")

            collected_successfully = False
            salary_estimate = -1
            location = -1
            job_title = -1
            company_name = -1
            job_description = -1
            salary_estimate = -1
            while not collected_successfully:
                try:
                    # En los siguientes textos vacíos vamos a rellenarlos con las clases de los
                    # elementos HTML que nos interese para recuperar la información
                    location = driver.find_element(By.CLASS_NAME,'JobDetails_location__mSg5h').text
                    job_title = driver.find_element(By.CLASS_NAME,'JobCard_jobTitle___7I6y').text
                    company_name = driver.find_element(By.CLASS_NAME,'heading_Subhead__Ip1aW').text
                    job_description = driver.find_element(By.CLASS_NAME,'JobDetails_jobDescriptionWrapper___tqxc').text
                    salary_estimate = driver.find_element(By.CLASS_NAME,'SalaryEstimate_salaryRange__brHFy').text
                    collected_successfully = True
                    print("job info collected successfully :)")
                except:
                    time.sleep(slp_time)
                    print("job info error, trying again...")
                    break

            job_data_dict = {
                            # "Job Title" : job_title,
                            "Location" : location,
                            "Company" : company_name,
                            "Salary Estimate" : salary_estimate,
                            "Job Description" : job_description,
                        }

            #Printing for debugging
            if verbose:
                print(job_data_dict)

            try:
                Company_Overview = driver.find_elements(By.CLASS_NAME,'JobDetails_overviewItemValue__xn8EF')
                company_overview_dict = get_company_overview_data(Company_Overview)

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                company_overview_dict = {"size":-1,
                                        "founded":-1,
                                        "type":-1,
                                        "industry":-1,
                                        "sector":-1,
                                        "revenue":-1}

            if verbose:
                print(company_overview_dict)
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            job_data_dict.update(company_overview_dict)

            #add job to jobs
            jobs.append(job_data_dict)

        jobs
    driver.close()
    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [42]:
path = "./chromedriver"
keyword = "Data Visualization"
num_jobs = 20
slp_time = 3
verbose = True

df = get_jobs(keyword, num_jobs, verbose, path, slp_time)
df

/var/folders/tl/_gc7cws53slcfj7kmnkdymgr0000gn/T/ipykernel_29603/2439416938.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driverpath)


job info collected successfully :)
{'Location': 'Winston-Salem, NC', 'Company': 'Wake Forest Baptist Health', 'Salary Estimate': '$59K – $89K/yr (Glassdoor est.)', 'Job Description': "Per candidates competency & experience leveling may be reviewed for placement alignment within WFUSM's CTSI department structure.\nThe DIVE team (Data, Integration, Visualization, Evaluation) is seeking a Programmer/Analyst skilled in statistical analysis , data visualization , and data science techniques. The DIVE team applies data science and informatics techniques to data problems arising in all facets of the academic enterprise, from education to research to strategic initiatives. This role will aim to drive growth and maintenance of education and research data warehouses and analysis pipelines, academic applications, and the informatics components of strategic projects. With multiple strategic combinations drastically increasing the scope of data and services managed by Informatics, this position wil

,Location,Company,Salary Estimate,Job Description,size,founded,type,industry,sector,revenue
0,"Winston-Salem, NC",Wake Forest Baptist Health,$59K – $89K/yr (Glassdoor est.),Per candidates competency & experience levelin...,10000+ Employees,1943,Hospital,Health Care Services & Hospitals,Healthcare,$1 to $5 billion (USD)
1,"Bowie, MD",Inovalon,$72K – $100K/yr (Employer est.),Inovalon was founded in 1998 on the belief tha...,1001 to 5000 Employees,1998,Company - Private,Software Development,Information Technology,$500 million to $1 billion (USD)
2,"Newtown Square, PA",Main Line Health,$89K – $139K/yr (Employer est.),Description:\nCould you be our next Data Analy...,10000+ Employees,1985,Nonprofit Organization,Health Care Services & Hospitals,Healthcare,$1 to $5 billion (USD)
3,Maryland,SSI,$100K – $130K/yr (Employer est.),Description:\nSSI is seeking an experienced Da...,1 to 50 Employees,2007,Company - Private,Information Technology Support Services,Information Technology,$1 to $5 million (USD)
4,"Norco, CA",DT Professional Services,$125K – $145K/yr (Employer est.),Job Summary:\n\nDT Professional Services is se...,1 to 50 Employees,2013,Company - Public,--,--,Less than $1 million (USD)
5,"Birmingham, AL",UAB Medicine,$64K – $104K/yr (Employer est.),"Compensation: $63,875 - $103,800\nGeneral Resp...",10000+ Employees,1945,Hospital,Health Care Services & Hospitals,Healthcare,Unknown / Non-Applicable
6,United States,Evolve Vacation Rentals,$200K – $230K/yr (Employer est.),At Evolve we’re a hardworking team serious abo...,501 to 1000 Employees,2011,Company - Private,Hotels & Resorts,Hotels & Travel Accommodation,Unknown / Non-Applicable
7,United States,Health Intelligence company LLC,-1,About us:\n\nBHI is a data analytics and softw...,1001 to 5000 Employees,2002,Unknown,Health Care Services & Hospitals,Healthcare,$100 to $500 million (USD)
8,"Brentwood, TN","Monogram Health, Inc",$60K – $100K/yr (Employer est.),Monogram is seeking a Healthcare Sr. Presentat...,501 to 1000 Employees,2019,Company - Private,Health Care Services & Hospitals,Healthcare,$1 to $5 billion (USD)
9,"West Orange, NJ",RWJBarnabas Health,-1,Job Overview:\nBusiness Intelligence (BI) tool...,10000+ Employees,1982,Nonprofit Organization,Health Care Services & Hospitals,Healthcare,Unknown / Non-Applicable
